# Langchain

In [35]:
import requests
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType

# Simulation d'un appelle à une API météo
def get_weather(city):
    temp = 25
    wind = 30
    return f"Il fait {temp}°C avec un vent à {wind} km/h."

# On crée le Tool LangChain
weather_tool = Tool.from_function(
    func=get_weather,
    name="get_weather",
    description="Donne la météo actuelle pour une ville spécifiée."
)

# On instancie ton modèle Ollama
#llm = ChatOllama(model="gemma3n:e2b")
#llm = ChatOllama(model="qwen2.5:3b")
llm = ChatOllama(model="phi4-mini:latest")

# On initialise l'agent
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prefix": """Si tu peux répondre directement à la question (sans appeler un outil), fais-le. 
N’utilise un outil que si c’est strictement nécessaire. 
Respecte scrupuleusement ce format pour les outils :

Thought: Raisonne brièvement
Action: nom_de_l_outil
Action Input: valeur"""}
)

# On pose une question à l'agent
result = agent.run("Peux-tu me dire la météo actuelle à Paris ?")
print(result)



> Entering new AgentExecutor chain...
Thought: To provide current weather information for Paris, I'll use a weather service tool.
Action: get_weather
Action Input: Paris

Observation: Il fait 25°C avec un vent à 30 km/h.
Thought:Final Answer: La température à Paris est de 25°C et le vent souffle à une vitesse de 30 km/h.

> Finished chain.
La température à Paris est de 25°C et le vent souffle à une vitesse de 30 km/h.


In [36]:
result = agent.run("Combien font 2+2 ?")
print(result)



> Entering new AgentExecutor chain...
The operation in this case is a basic arithmetic addition, which doesn't require any external tools or APIs. The result of adding two and two together can be directly computed.

Action: I already know how
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:Question: Combien font 2+2?
Thought: This is a simple arithmetic question that doesn't require any external tools or APIs. The result of adding two and two together can be directly computed.

Final Answer: 2 + 2 equals 4.

> Finished chain.
2 + 2 equals 4.


Analyse :
Beaucoup de problémes soit il utilise trop l'outils soit il y a des erreurs de parsing. Pas très stable. Ou alors la réponse est en anglais.